# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh our knowledge of SQL Queries!

## Objectives

You will be able to:

- Practice your SQL knowledge

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as `pandas`. By combining them, we'll be able to write our queries as python strings, and make sure that the results are always returned as a pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3`
* Establish a connection to the database `data.sqlite`
* Get the `cursor` from the connection and store it in the variable `c`.

In [1]:
import pandas as pd
import sqlite3
conn = sqlite3.Connection("data.sqlite")
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

* Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [2]:
# For the first query, the boilerplate for getting 
#the query into a dataframe has been provided for you
c.execute(""" select contactLastName, contactFirstName, phone, 
                addressLine1, city, state, postalCode, creditLimit
              from customers
              where state == "CA"
              and creditLimit > 25000;
          """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,contactLastName,contactFirstName,phone,addressLine1,city,state,postalCode,creditLimit
0,Nelson,Susan,4155551450,5677 Strong St.,San Rafael,CA,97562,210500.0
1,Murphy,Julie,6505555787,5557 North Pendale Street,San Francisco,CA,94217,64600.0
2,Hashimoto,Juri,6505556809,9408 Furth Circle,Burlingame,CA,94217,84600.0
3,Young,Julie,6265557265,78934 Hillside Dr.,Pasadena,CA,90003,90700.0
4,Thompson,Valarie,7605558146,361 Furth Circle,San Diego,CA,91217,105000.0
5,Brown,Julie,6505551386,7734 Strong St.,San Francisco,CA,94217,105000.0
6,Chandler,Brian,2155554369,6047 Douglas Av.,Los Angeles,CA,91003,57700.0
7,Frick,Sue,4085553659,3086 Ingle Ln.,San Jose,CA,94217,77600.0
8,Thompson,Steve,3105553722,3675 Furth Circle,Burbank,CA,94019,55400.0
9,Taylor,Sue,4155554312,2793 Furth Circle,Brisbane,CA,94217,60300.0


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that get sthe average credit limit per state.

In [5]:
c.execute(""" select state, AVG(creditLimit) 
                from customers
                group by state;
          """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,state,AVG(creditLimit)
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000
5,Isle of Wight,93900.000000
6,MA,70755.555556
7,NH,114200.000000
8,NJ,43000.000000
9,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [6]:
c.execute(""" select c.customerNumber, c.customerName,
                o.orderNumber, o.status, od.quantityOrdered 
            from customers c 
            join orders o ON c.customerNumber = o.customerNumber
            join orderdetails od ON od.orderNumber = o.orderNumber;
          """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,customerName,orderNumber,status,quantityOrdered
0,363,Online Diecast Creations Co.,10100,Shipped,22
1,363,Online Diecast Creations Co.,10100,Shipped,30
2,363,Online Diecast Creations Co.,10100,Shipped,49
3,363,Online Diecast Creations Co.,10100,Shipped,50
4,128,"Blauer See Auto, Co.",10101,Shipped,25


#### Expected Output

<img src='images/joins.png'>

## HAVING and ORDER BY

Now, return the customerName, customrerNumber, productName, productCode and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY--make sure you get the order of them correct!

In [31]:
c.execute(""" select c.customerNumber, c.customerName, sum(od.quantityOrdered) as TTLNumOrd,
                p.productName, p.productCode 
            from customers c 
            join orders o ON c.customerNumber = o.customerNumber
            join orderdetails od ON od.orderNumber = o.orderNumber
            join products p  ON p.productCode = od.productcode
            group by o.customerNumber, od.productcode
            having sum(od.quantityOrdered) >= 10
            order by TTLNumOrd DESC;
          """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerNumber,customerName,TTLNumOrd,productName,productCode
0,141,Euro+ Shopping Channel,308,1992 Ferrari 360 Spider red,S18_3232
1,141,Euro+ Shopping Channel,245,1958 Chevy Corvette Limited Edition,S24_2840
2,141,Euro+ Shopping Channel,197,1970 Dodge Coronet,S24_1444
3,141,Euro+ Shopping Channel,183,1957 Chevy Pickup,S12_4473
4,141,Euro+ Shopping Channel,174,2002 Chevy Corvette,S24_3432


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from an office with less than 5 employees. 

In [32]:
c.execute(""" select firstName, lastName, employeeNumber, officeCode 
              from employees
              where officeCode in ( select officeCode from offices 
                                      join employees using (officeCode)
                                      group by officecode
                                      having count (employeeNumber) < 5);
          """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,firstName,lastName,employeeNumber,officeCode
0,William,Patterson,1088,6
1,Julie,Firrelli,1188,2
2,Steve,Patterson,1216,2
3,Foon Yue,Tseng,1286,3
4,George,Vanauf,1323,3


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!